In [101]:
import pandas as pd
from math import log10

df_nirvana = pd.read_csv('Dados/nirvana_lyrics.csv')
df_pink_floyd = pd.read_csv('Dados\pink_floyd_lyrics.csv')

In [102]:
df_nirvana_treino = df_nirvana.iloc[:50]
df_pink_floyd_treino = df_pink_floyd.iloc[:50]

df_nirvana_teste = df_nirvana.iloc[50:100]
df_pink_floyd_teste = df_pink_floyd.iloc[50:100]

In [103]:
probabilidade_nirvana = len(df_nirvana_treino) / (len(df_nirvana_treino) + len(df_pink_floyd_treino))
probabilidade_pink_floyd = len(df_pink_floyd_treino) / (len(df_nirvana_treino) + len(df_pink_floyd_treino))
probabilidade_total = probabilidade_nirvana + probabilidade_pink_floyd

In [104]:
import re
def arruma(lyrics):
    if isinstance(lyrics, str):
        lyrics = lyrics.lower()
        lyrics = re.sub(r"[.!@#$%&*(){}:;_?/’<>+|=,\[\]\n']", '', lyrics)
        lyrics = re.sub(r'[\"]', '', lyrics)


    return lyrics

def split_function(texto):
    return texto.split()

df_nirvana_treino.loc[:,'lyrics'] = df_nirvana_treino['lyrics'].astype(str)
df_pink_floyd_treino.loc[:,'lyrics'] = df_pink_floyd_treino['lyrics'].astype(str)

df_nirvana_treino.loc[:,'lyrics'] = df_nirvana_treino['lyrics'].apply(arruma)
df_pink_floyd_treino.loc[:,'lyrics'] = df_pink_floyd_treino['lyrics'].apply(arruma)

df_nirvana_treino = pd.DataFrame(df_nirvana_treino)
df_pink_floyd_treino = pd.DataFrame(df_pink_floyd_treino)

def keep_unique_words(text):
    words = text.split()
    unique_words = set(words)
    return ' '.join(unique_words)

df_nirvana_treino['lyrics'] = df_nirvana_treino['lyrics'].apply(keep_unique_words)
df_pink_floyd_treino['lyrics'] = df_pink_floyd_treino['lyrics'].apply(keep_unique_words)


In [105]:
nirvaninha = {}
pinkinho = {}
totalzinho = {}
contador1 = 0
contador2 = 0

for musicas_n in df_nirvana_treino['lyrics']:
    nirvaninha[contador1] = musicas_n.split()
    totalzinho[contador1] = musicas_n.split()
    contador1 += 1

for musicas_pf in df_pink_floyd_treino['lyrics']:
    pinkinho[contador2] = musicas_pf.split()
    totalzinho[contador2+50] = musicas_pf.split()
    contador2 += 1

print(pinkinho)
print(nirvaninha)
print(totalzinho)

{0: ['stars', 'was', 'fight', 'both', 'oberon', 'blampow', 'flicker', 'discovered', 'signs', 'underground', 'knewfloating', 'not', 'green', 'powstairway', 'once', 'limpid', 'the', 'waters', 'flapflicker', 'underlime', 'saturn', 'icy', 'you', 'until', 'titanianeptune', 'down', 'moon', 'second', 'between', 'frightenblinding', 'whos', 'scare', 'skies', 'in', 'scenea', 'titan', 'dan', 'dare', 'housesscorpio', 'sound', 'surroundsthe', 'arabian', 'blue', '1930lime', 'resoundsaround', 'miranda', 'therelime', 'librapluto', 'undergroundjupiter', 'can', 'a', 'and'], 1: ['sam', 'ships', 'found', 'catalways', 'something', 'cats', 'groundhell', 'sandhiding', 'by', 'hip', 'right', 'explain', 'on', 'side', 'sideoh', 'hes', 'the', 'sifting', 'to', 'anywherethat', 'nothat', 'be', 'sidethat', 'prowling', 'left', 'siam', 'when', 'night', 'sitting', 'explainjennifer', 'witchyoure', 'explainat', 'your', 'gentle', 'go', 'aroundthat', 'seabe', 'youre', 'catsomewhere', 'cant', 'explainlucifer', 'around', 'luc

In [106]:
frequencia_nirvana = {}
frequencia_pink_floyd = {}
frequencia_total = {}

for musicas in nirvaninha.values():
    for palavras in musicas:
        if palavras in frequencia_nirvana:
            frequencia_nirvana[palavras] += 1
        else:
            frequencia_nirvana[palavras] = 1

for musicas in pinkinho.values():
    for palavras in musicas:
        if palavras in frequencia_pink_floyd:
            frequencia_pink_floyd[palavras] += 1
        else:
            frequencia_pink_floyd[palavras] = 1

for musicas in totalzinho.values():
    for palavras in musicas:
        if palavras in frequencia_total:
            frequencia_total[palavras] += 1
        else:
            frequencia_total[palavras] = 1

print(frequencia_pink_floyd)
print(frequencia_nirvana)
print(frequencia_total)

{'stars': 2, 'was': 8, 'fight': 2, 'both': 3, 'oberon': 2, 'blampow': 2, 'flicker': 2, 'discovered': 2, 'signs': 2, 'underground': 2, 'knewfloating': 2, 'not': 7, 'green': 5, 'powstairway': 2, 'once': 4, 'limpid': 2, 'the': 36, 'waters': 2, 'flapflicker': 2, 'underlime': 2, 'saturn': 2, 'icy': 2, 'you': 21, 'until': 2, 'titanianeptune': 2, 'down': 9, 'moon': 4, 'second': 2, 'between': 4, 'frightenblinding': 2, 'whos': 3, 'scare': 2, 'skies': 2, 'in': 31, 'scenea': 2, 'titan': 2, 'dan': 2, 'dare': 2, 'housesscorpio': 2, 'sound': 5, 'surroundsthe': 2, 'arabian': 2, 'blue': 3, '1930lime': 2, 'resoundsaround': 2, 'miranda': 2, 'therelime': 2, 'librapluto': 2, 'undergroundjupiter': 2, 'can': 9, 'a': 30, 'and': 23, 'sam': 1, 'ships': 1, 'found': 2, 'catalways': 1, 'something': 3, 'cats': 1, 'groundhell': 1, 'sandhiding': 1, 'by': 11, 'hip': 1, 'right': 2, 'explain': 1, 'on': 18, 'side': 2, 'sideoh': 1, 'hes': 4, 'sifting': 1, 'to': 26, 'anywherethat': 1, 'nothat': 1, 'be': 11, 'sidethat': 1,

In [107]:
probabilidade_por_palavra_nirvana = {}
probabilidade_por_palavra_pink_floyd = {}
probabilidade_por_palavra_total = {}

for palavra,quantidade in frequencia_nirvana.items():
    probabilidade_por_palavra_nirvana[palavra] = quantidade/len(nirvaninha)

for palavra,quantidade in frequencia_pink_floyd.items():
    probabilidade_por_palavra_pink_floyd[palavra] = quantidade/len(pinkinho)

for palavra,quantidade in frequencia_total.items():
    probabilidade_por_palavra_total[palavra] = quantidade/len(totalzinho)

print(probabilidade_por_palavra_pink_floyd)
print(probabilidade_por_palavra_nirvana)
print(probabilidade_por_palavra_total)

{'stars': 0.04, 'was': 0.16, 'fight': 0.04, 'both': 0.06, 'oberon': 0.04, 'blampow': 0.04, 'flicker': 0.04, 'discovered': 0.04, 'signs': 0.04, 'underground': 0.04, 'knewfloating': 0.04, 'not': 0.14, 'green': 0.1, 'powstairway': 0.04, 'once': 0.08, 'limpid': 0.04, 'the': 0.72, 'waters': 0.04, 'flapflicker': 0.04, 'underlime': 0.04, 'saturn': 0.04, 'icy': 0.04, 'you': 0.42, 'until': 0.04, 'titanianeptune': 0.04, 'down': 0.18, 'moon': 0.08, 'second': 0.04, 'between': 0.08, 'frightenblinding': 0.04, 'whos': 0.06, 'scare': 0.04, 'skies': 0.04, 'in': 0.62, 'scenea': 0.04, 'titan': 0.04, 'dan': 0.04, 'dare': 0.04, 'housesscorpio': 0.04, 'sound': 0.1, 'surroundsthe': 0.04, 'arabian': 0.04, 'blue': 0.06, '1930lime': 0.04, 'resoundsaround': 0.04, 'miranda': 0.04, 'therelime': 0.04, 'librapluto': 0.04, 'undergroundjupiter': 0.04, 'can': 0.18, 'a': 0.6, 'and': 0.46, 'sam': 0.02, 'ships': 0.02, 'found': 0.04, 'catalways': 0.02, 'something': 0.06, 'cats': 0.02, 'groundhell': 0.02, 'sandhiding': 0.02

In [108]:
def teorema_de_bayes(musica):
    musica = musica.split()
    propabilidade_final_pf = 0
    propabilidade_final_n = 0

    for palavras in musica:
        if palavras in probabilidade_por_palavra_pink_floyd:
            soma_pf = log10(probabilidade_por_palavra_pink_floyd[palavras])
            subtracao_pf = log10(probabilidade_por_palavra_total[palavras])
            propabilidade_final_pf += soma_pf
            propabilidade_final_pf -= subtracao_pf
    propabilidade_final_pf *= probabilidade_pink_floyd

    for palavras in musica:
        if palavras in probabilidade_por_palavra_nirvana:
            soma_n = log10(probabilidade_por_palavra_nirvana[palavras])
            subtracao_n = log10(probabilidade_por_palavra_total[palavras])
            propabilidade_final_n += soma_n
            propabilidade_final_n -= subtracao_n
    propabilidade_final_n *= probabilidade_nirvana
    
    return propabilidade_final_pf, propabilidade_final_n


In [109]:
music = '''
'''

final_pf, final_n = teorema_de_bayes(music)
print(final_pf)
print(final_n)
if final_pf > final_n:
    print("Essa música é do Pink Floyd")
elif final_n > final_pf:
    print("Essa música é do Nirvana")

-1.4087665942891265
2.908931995041182
Essa música é do Nirvana
